In [1]:
from N_Gram import N_Gram
import numpy as np
import csv
import random
from Linear_Regression import model

#### 创建语言模型

In [2]:
with open('../data/train.tsv') as train_file:
    reader = csv.reader(train_file , delimiter='\t')
    data = list(reader)
data = data[1:] # 有表头

train_feature = N_Gram(data , dimension=4)
train_feature.create_dict()

create gram dict : 4-gram: 100%|██████████| 156060/156060 [00:00<00:00, 406406.94it/s]


#### 对测试数据进行特征提取

In [3]:
with open('../data/test.tsv') as test_file:
    reader = csv.reader(test_file , delimiter='\t')
    test_data = list(reader)
test_data = test_data[1:] # 有表头

test_feature = N_Gram(test_data , dimension=4)
test_feature.word_dict = train_feature.word_dict
test_feature.dict_len = train_feature.dict_len
test_phrase = [item[2] for item in test_data]

In [4]:
train_feature.dict_len

360247

#### 读取模型数据

In [5]:
classifier = model(5 , train_feature.dict_len)
tmp_list = []
model_name = '2023-03-12 10_38-model'


with open('../model/'+model_name+'.tsv' , 'r') as trained_model:
    for line in trained_model:
        tmp_list.append(line.strip().split('\t'))
classifier.W_matrix = np.array(tmp_list, dtype=np.float64)

classifier.W_matrix.shape

(360247, 5)

#### 开始预测(直接使用所有进行矩阵计算内存不够算)

In [6]:
result = []

for phrase in test_phrase:
    tmp_feature = np.zeros((1,test_feature.dict_len),dtype=np.float64)
    #获取一行的特征表示
    for dime in range(1,5):
        words = phrase.lower().split()
        for j in range(len(words)-dime+1):
            tokenized_word = ' '.join(words[j : j+dime])
            if tokenized_word in test_feature.word_dict:
                tmp_feature[0][test_feature.word_dict[tokenized_word]] = 1
    #开始计算预测
    result.append(classifier.prediction(tmp_feature))

#### 准备提交

In [7]:
phraseId = [item[0] for item in test_data]
submit_data = [[x,y[0]] for x,y in zip(phraseId , result)]

In [8]:
import pandas as pd

df = pd.DataFrame(submit_data, columns=['PhraseId', 'Sentiment'])
df.to_csv(f'{model_name}-submit.csv', index=None)